In [ ]:
import pandas as pd
import numpy as np
import pandas_ta as ta
import plotly.graph_objects as go
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score

# Load data
data = pd.read_csv("RNDRUSDT 15m.csv", parse_dates=['Date'], index_col='Date')

In [ ]:
# Calculate VWAP
data.ta.vwap(append=True)

Date
2024-05-24 00:00:00    10.260333
2024-05-24 00:15:00    10.254074
2024-05-24 00:30:00    10.254858
2024-05-24 00:45:00    10.257511
2024-05-24 01:00:00    10.253774
                         ...    
2024-06-02 23:00:00     9.927141
2024-06-02 23:15:00     9.927059
2024-06-02 23:30:00     9.926968
2024-06-02 23:45:00     9.926968
2024-06-03 00:00:00     9.943000
Name: VWAP_D, Length: 961, dtype: float64

In [ ]:
# Define the check_candles function with additional parameter optimization
def check_candles(data, backcandles, ma_column):
    categories = [0 for _ in range(backcandles)]
    for i in range(backcandles, len(data)):
        if all(data['close'][i-backcandles:i] > data[ma_column][i-backcandles:i]):
            categories.append(2)  # Uptrend
        elif all(data['close'][i-backcandles:i] < data[ma_column][i-backcandles:i]):
            categories.append(1)  # Downtrend
        else:
            categories.append(0)  # No trend
    return categories

In [ ]:
# Optimize parameters
best_params = None
best_score = -1

In [ ]:
# Define the parameter grid for optimization
param_grid = {    'backcandles': [3, 5, 7],
    'vwap_period': [20, 30, 50]
} 

for params in ParameterGrid(param_grid):
    # Calculate VWAP with different periods
    data['VWAP'] = data['close'].rolling(window=params['vwap_period']).mean()
    
    # Apply the check_candles function
    data['Category'] = check_candles(data, params['backcandles'], 'VWAP')
    
    # Evaluate performance (for simplicity, we use a dummy accuracy score)
    score = accuracy_score(data['Category'][params['backcandles']:], 
                           data['Category'][params['backcandles']:])
    
    if score > best_score:
        best_score = score
        best_params = params

print(f"Best parameters: {best_params} with score: {best_score}")

Best parameters: {'backcandles': 3, 'vwap_period': 20} with score: 1.0


In [ ]:
data

,open,high,low,close,volume,adj close,VWAP_D,VWAP,Category
Date,,,,,,,,,
2024-05-24 00:00:00,10.234,10.342,10.201,10.238,136035.18,1.716510e+12,10.260333,NaN,0
2024-05-24 00:15:00,10.237,10.264,10.205,10.229,39778.10,1.716511e+12,10.254074,NaN,0
2024-05-24 00:30:00,10.229,10.291,10.222,10.265,30801.89,1.716511e+12,10.254858,NaN,0
2024-05-24 00:45:00,10.265,10.293,10.243,10.273,45102.33,1.716512e+12,10.257511,NaN,0
2024-05-24 01:00:00,10.272,10.272,10.192,10.239,48385.04,1.716513e+12,10.253774,NaN,0
...,...,...,...,...,...,...,...,...,...
2024-06-02 23:00:00,9.892,9.901,9.891,9.896,8173.44,1.717370e+12,9.927141,9.92998,1
2024-06-02 23:15:00,9.896,9.900,9.891,9.895,7979.03,1.717371e+12,9.927059,9.92932,1
2024-06-02 23:30:00,9.895,9.917,9.892,9.911,13908.07,1.717372e+12,9.926968,9.92946,1


In [ ]:
# Apply the best parameters
data['VWAP'] = data['close'].rolling(window=best_params['vwap_period']).mean()
data['Category'] = check_candles(data, best_params['backcandles'], 'VWAP')

In [ ]:
# Combine with other indicators (RSI and MACD)
data['RSI'] = ta.rsi(data['close'], length=14)
macd = ta.macd(data['close'])
data = pd.concat([data, macd], axis=1)

In [ ]:
# Filter data for potential buy/sell signals
buy_signals = data[(data['Category'] == 2) & (data['RSI'] < 30) & (data['MACD_12_26_9'] > data['MACDs_12_26_9'])]
sell_signals = data[(data['Category'] == 1) & (data['RSI'] > 70) & (data['MACD_12_26_9'] < data['MACDs_12_26_9'])]

print(f"Buy signals:\n{buy_signals}")
print(f"Sell signals:\n{sell_signals}")

Buy signals:
Empty DataFrame
Columns: [open, high, low, close, volume, adj close, VWAP_D, VWAP, Category, RSI, MACD_12_26_9, MACDh_12_26_9, MACDs_12_26_9]
Index: []
Sell signals:
Empty DataFrame
Columns: [open, high, low, close, volume, adj close, VWAP_D, VWAP, Category, RSI, MACD_12_26_9, MACDh_12_26_9, MACDs_12_26_9]
Index: []


In [ ]:
data

,open,high,low,close,volume,adj close,VWAP_D,VWAP,Category,RSI,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
Date,,,,,,,,,,,,,
2024-05-24 00:00:00,10.234,10.342,10.201,10.238,136035.18,1.716510e+12,10.260333,NaN,0,NaN,NaN,NaN,NaN
2024-05-24 00:15:00,10.237,10.264,10.205,10.229,39778.10,1.716511e+12,10.254074,NaN,0,NaN,NaN,NaN,NaN
2024-05-24 00:30:00,10.229,10.291,10.222,10.265,30801.89,1.716511e+12,10.254858,NaN,0,NaN,NaN,NaN,NaN
2024-05-24 00:45:00,10.265,10.293,10.243,10.273,45102.33,1.716512e+12,10.257511,NaN,0,NaN,NaN,NaN,NaN
2024-05-24 01:00:00,10.272,10.272,10.192,10.239,48385.04,1.716513e+12,10.253774,NaN,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-02 23:00:00,9.892,9.901,9.891,9.896,8173.44,1.717370e+12,9.927141,9.87570,2,48.631858,-0.004731,0.006508,-0.011240
2024-06-02 23:15:00,9.896,9.900,9.891,9.895,7979.03,1.717371e+12,9.927059,9.88015,2,48.432181,-0.004544,0.005356,-0.009900
2024-06-02 23:30:00,9.895,9.917,9.892,9.911,13908.07,1.717372e+12,9.926968,9.88240,2,51.839431,-0.003070,0.005465,-0.008534


In [ ]:
# Visualization
dfpl = data[:]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                                     open=dfpl['open'],
                                     high=dfpl['high'],
                                     low=dfpl['low'],
                                     close=dfpl['close'])])

fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['VWAP'], mode='lines', name='VWAP', line=dict(color='red')))
fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['close'], mode='markers', name='Buy Signal', 
                         marker=dict(color='green', size=10, symbol='triangle-up')))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['close'], mode='markers', name='Sell Signal', 
                         marker=dict(color='red', size=10, symbol='triangle-down')))

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()